In [1]:
import tensorflow as tf
import numpy as np

In [2]:
Batch_Size = 5
feature_size = 9
def get_feature_label(filenames_list):
    filename_queue = tf.train.string_input_producer(filenames_list)
    reader = tf.TextLineReader(skip_header_lines=1)
    key,value = reader.read(filename_queue)
    record_defaults = [[1.0] for i in range(9)]
    record_defaults[4] = ['']
    record_defaults.append([1])
    content = tf.decode_csv(value,record_defaults=record_defaults)
    condition = tf.equal(content[4],tf.constant('Present'))
    content[4] = tf.select(condition,tf.constant(1.0),tf.constant(0.0))
    features = tf.pack(content[:feature_size])
    label = content[-1]


    minimum_after_dequeue = 10*Batch_Size
    #capacity: Recommendation:   min_after_dequeue + 
    #(num_threads + a small safety margin) * batch_size
    capacity = 1.5*minimum_after_dequeue
    data_batch,label_batch = tf.train.shuffle_batch([features,label],batch_size=Batch_Size,
                                                   capacity=capacity,min_after_dequeue=minimum_after_dequeue)
    #label_batch = tf.reshape(label_batch,[Batch_Size,1])
    return data_batch,label_batch


In [3]:
X = tf.placeholder(tf.float32,[Batch_Size,feature_size],name='X_placeholder')
Y = tf.placeholder(tf.int32,[Batch_Size],name='Y_placeholder')
learning_rate = tf.placeholder(tf.float32)
epsilon= tf.placeholder(tf.float32)
Y_one_hot = tf.one_hot(Y,2)
# w = tf.Variable([[0.1] for _ in range(9)],name="Weights")
w = tf.Variable(tf.truncated_normal(shape=[feature_size,2],stddev=0.1),name="Weights")
b = tf.Variable(tf.zeros([1, 2]),name="biases")
logits = tf.matmul(X,w) + b

entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot,name='loss')
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,epsilon=epsilon).minimize(loss)

In [4]:
data_batch,label_batch = get_feature_label(["heart_preprocessed.csv"])

In [5]:
final_features = []
final_labels = []
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #print sess.run(key)
    #data_batch,label_batch = sess.run([data_batch,label_batch])
    for _ in range(90):
        features,labels = sess.run([data_batch,label_batch])
        final_features.append(features)
        final_labels.append(labels)
    coord.request_stop()
    coord.join(threads)

In [6]:
print len(final_labels),len(final_features)

90 90


In [7]:
print tf.Session().run(tf.equal(final_features[0][0],final_features[-1][0]))

[False False False False False False False False False]


In [11]:
saver = tf.train.Saver()

with tf.Session() as sess:
    
    learning_rate_list = [1e-2]
    epsilon_list = [1e-10]
#     learning_rate_list = [1e-1,1e-2,1e-4,1e-6]
#     epsilon_list = [1e-7,1e-8,1e-9,1e-10] 
    accuracies = {}
    sess.run(tf.global_variables_initializer())
    for lr in learning_rate_list:
        for ep in epsilon_list:
            for epoch in range(50):
                total_loss = 0
                test_count = 0
                train_count = 0
                total_correct_preds = 0
                #print len(final_features)
                for i in range(len(final_features)):
                    temp_labels = final_labels[i]
        #             temp_labels = np.reshape(temp_labels,(-1,1))
                    temp_features = final_features[i]
                    if i<=80: 
                        train_count+=len(temp_features)
                        _,l,weight,bias = sess.run([optimizer,loss,w,b],feed_dict={X:temp_features,Y:temp_labels,learning_rate:lr,epsilon:ep})
                        total_loss+=l
                    else:
                        test_count+=len(temp_features)
                        _,l,Y_predicted= sess.run([optimizer,loss,logits],feed_dict={X:temp_features,Y:temp_labels,learning_rate:lr,epsilon:ep})
                        preds_prob = tf.nn.softmax(Y_predicted)
                        max_preds = tf.argmax(preds_prob,1)
                        correct_preds = tf.equal(max_preds,temp_labels)
                        non_zero = tf.count_nonzero(correct_preds)
                        accuracy = tf.reduce_sum(non_zero)
                        total_correct_preds += sess.run(accuracy)
                        #print sess.run(max_preds),temp_labels,sess.run(accuracy),sess.run(non_zero)
                print 'Accuracy at the epoch {0} is {1}'.format(epoch,total_correct_preds/float(test_count))
                print 'Loss at the epoch {0} is {1}'.format(epoch,total_loss/float(train_count))
                #accuracies[str(lr)+'_'+str(ep)+'_'+str(epoch)] = [total_correct_preds/float(test_count),total_loss/float(train_count)]
                #print str(lr)+'_'+str(ep)+'_'+str(epoch),accuracies[str(lr)+'_'+str(ep)+'_'+str(epoch)]
                
            save_path = saver.save(sess, str(lr)+'_'+str(ep)+"_model.ckpt")
            print save_path

Accuracy at the epoch 0 is 0.777777777778
Loss at the epoch 0 is 0.124560913848
Accuracy at the epoch 1 is 0.822222222222
Loss at the epoch 1 is 0.111926896962
Accuracy at the epoch 2 is 0.844444444444
Loss at the epoch 2 is 0.109529307227
Accuracy at the epoch 3 is 0.866666666667
Loss at the epoch 3 is 0.108700987108
Accuracy at the epoch 4 is 0.866666666667
Loss at the epoch 4 is 0.108397977256
Accuracy at the epoch 5 is 0.866666666667
Loss at the epoch 5 is 0.108292233613
Accuracy at the epoch 6 is 0.866666666667
Loss at the epoch 6 is 0.108262794511
Accuracy at the epoch 7 is 0.866666666667
Loss at the epoch 7 is 0.108262350751
Accuracy at the epoch 8 is 0.866666666667
Loss at the epoch 8 is 0.108271950851
Accuracy at the epoch 9 is 0.866666666667
Loss at the epoch 9 is 0.108284052323
Accuracy at the epoch 10 is 0.866666666667
Loss at the epoch 10 is 0.10829580062
Accuracy at the epoch 11 is 0.866666666667
Loss at the epoch 11 is 0.1083062637
Accuracy at the epoch 12 is 0.866666666

In [ ]:
#1e-2,1e-10 = 86.7% 0.01_1e-10_model.ckpt is the best model